In [1]:
import ipaddress

# Department host requirements
departments = {
    'Engineering': 30,
    'Marketing': 15,
    'Finance': 10,
    'HR': 5
}

# Function to calculate the smallest subnet mask that fits required hosts
def get_subnet_prefix(required_hosts):
    for prefix in range(32, 0, -1):
        usable_hosts = 2 ** (32 - prefix) - 2
        if usable_hosts >= required_hosts:
            return prefix
    return None

# Start from base network
base_network = ipaddress.IPv4Network("172.16.0.0/16")
available_subnets = list(base_network.subnets(new_prefix=27))  # start with largest required

allocated_subnets = {}
current_index = 0

# Sort departments by descending host requirement (best-fit allocation)
for dept, hosts in sorted(departments.items(), key=lambda x: -x[1]):
    prefix = get_subnet_prefix(hosts)
    while True:
        candidate = available_subnets[current_index]
        if candidate.prefixlen <= prefix:
            # Subdivide until we match the prefix
            while candidate.prefixlen < prefix:
                candidate = list(candidate.subnets())[0]
            allocated_subnets[dept] = candidate
            current_index += 1
            break
        else:
            current_index += 1

# Display the subnetting plan
print("Subnetting Plan:\n")
for dept, subnet in allocated_subnets.items():
    hosts = list(subnet.hosts())
    print(f"Department: {dept}")
    print(f"  Subnet: {subnet}")
    print(f"  Subnet Mask: {subnet.netmask}")
    print(f"  Network Address: {subnet.network_address}")
    print(f"  Broadcast Address: {subnet.broadcast_address}")
    print(f"  Usable Host Range: {hosts[0]} - {hosts[-1]}")
    print(f"  Total Usable Hosts: {len(hosts)}\n")


Subnetting Plan:

Department: Engineering
  Subnet: 172.16.0.0/27
  Subnet Mask: 255.255.255.224
  Network Address: 172.16.0.0
  Broadcast Address: 172.16.0.31
  Usable Host Range: 172.16.0.1 - 172.16.0.30
  Total Usable Hosts: 30

Department: Marketing
  Subnet: 172.16.0.32/27
  Subnet Mask: 255.255.255.224
  Network Address: 172.16.0.32
  Broadcast Address: 172.16.0.63
  Usable Host Range: 172.16.0.33 - 172.16.0.62
  Total Usable Hosts: 30

Department: Finance
  Subnet: 172.16.0.64/28
  Subnet Mask: 255.255.255.240
  Network Address: 172.16.0.64
  Broadcast Address: 172.16.0.79
  Usable Host Range: 172.16.0.65 - 172.16.0.78
  Total Usable Hosts: 14

Department: HR
  Subnet: 172.16.0.96/29
  Subnet Mask: 255.255.255.248
  Network Address: 172.16.0.96
  Broadcast Address: 172.16.0.103
  Usable Host Range: 172.16.0.97 - 172.16.0.102
  Total Usable Hosts: 6

